In [1]:
import os
import pandas as pd


In [2]:
for file in os.listdir('.'):
    if 'claim' in file:
        data = pd.read_excel(file, skiprows=2, skipfooter=10, na_values=' ', usecols=['Дата создания претензии', \
                                                                        'Дата закрытия претензии',\
                                                                       'Претензионный менеджер'])
        break

In [3]:
data['Дата создания претензии'], data['Дата закрытия претензии'] = pd.to_datetime(data['Дата создания претензии'], format='%d.%m.%y'), pd.to_datetime(data['Дата закрытия претензии'], format='%d.%m.%y')

In [4]:
data = data[data['Дата закрытия претензии'].isin(pd.date_range(start='2020-09-01', end='2020-09-30'))].reset_index(drop=True)

In [5]:
data['Дней на закрытие'] = (data['Дата закрытия претензии'] - data['Дата создания претензии']).apply(lambda x: x.days)

In [6]:
group_data = data.groupby('Претензионный менеджер', as_index=False).agg({'Дата создания претензии' : 'count', 'Дней на закрытие' : 'mean'}).rename(columns = {'Дата создания претензии' : 'Закрыто обращений в сентябре', 'Дней на закрытие' : 'Среднее время закрытия обращения'})

In [7]:
group_data['Sort_index'] = group_data['Претензионный менеджер'].apply(lambda x: int(x.split()[-1]))

group_data = group_data.sort_values(by=['Sort_index'])


In [8]:
group_data.index = group_data['Претензионный менеджер']

group_data.drop(['Sort_index', 'Претензионный менеджер'], axis=1, inplace=True)

group_data.to_excel('Менеджеры_сентябрь.xlsx')